## Clause Detection Using Pre-Trained Model

In [ ]:
from tclp.clause_detector import detector_utils as du

In [ ]:
model_name = "../clause_identifier_model.pkl"
model = du.load_model(model_name)

In [ ]:
contract_dir = "../../data/test_contracts"
# NOTE: If this throws an error, it is because you may have uniquely constructed your test set from training_model.ipynb. 
# Because of randonmess, different contracts may be selected for training and testing. 
# Just go into the data/test_contracts directory and pull the name of a single contract to fix this.
single_contract = "../../data/synth_data/modified_real/000000930.txt"

In [ ]:
#you can input contract_dir or single_contract as the argument to this function
processed_contracts = du.load_unlabelled_contract(single_contract)

In [ ]:
processed_contracts

In [ ]:
results = model.predict(processed_contracts["text"])

In [ ]:
#note: This function is designed to work for single contracts. 
# It will work with a contract directory but they will all be in one, very long file 
highlighted_output = du.highlight_climate_content(processed_contracts["text"], results)

In [ ]:
#save highlighted output to a file; commented so it doesn't save every time this script runs 
#du.save_file("highlighted_output.html", highlighted_output)

In [ ]:
contract_df = du.create_contract_df(
    processed_contracts["text"], processed_contracts, results, labelled=False
)

In [ ]:
likely, very_likely, extremely_likely, none = du.create_threshold_buckets(contract_df)

In [ ]:
du.print_percentages(likely, very_likely, extremely_likely, none, contract_df)

In [ ]:
# note: this doesn't work if you are working with a contract database; it is designed for a single file
# In the way this file is currently written, this outputs nonsense because we are working with a folder
# of contracts, not a single contract.

# If you want to see this functionality, run the code using the single file path
du.print_single(likely, very_likely, extremely_likely, none)

I split the data 60% including a clause and 40% not including a clause so this looks good! 